# OpenPose Jupyter Data Exploration
## Imports and dependencies

In [1]:
import pandas as pd
from pandas import DataFrame

import plotly.express as px
import plotly.graph_objects as go
from glob import glob

from path import Path

Mandatory input:
* frame data path
* frame files count
* persons in frame count

## Import data

In [2]:
def get_frame_data(frame_file):
    try:
        frame_data = pd.read_csv(frame_file)
        
        return frame_data
    except FileNotFoundError:
        print(f"Frame data not found for frame {frame_idx}")  
        return pd.DataFrame()

In [3]:
def get_person_files(input_files, frame_data_path):
    all_persons_files = {}
    max_persons = 10
    
    for input_file_idx in range(len(input_files)):
        for person_idx in range(max_persons):
            expected_file = frame_data_path.replace("[frame_idx]", str(input_file_idx))
            expected_file = expected_file.replace("[person_idx]", str(person_idx))
            
            if expected_file in input_files:
                person_files = []
                
                if person_idx in all_persons_files:
                    person_files = all_persons_files[person_idx]
                
                person_files.append(expected_file)
                
                all_persons_files[person_idx] = person_files
                
    persons_indeces = all_persons_files.keys()
    print(f"Found {len(persons_indeces)} persons")
    for person_idx in persons_indeces:
        print(f"Person {person_idx} has {len(all_persons_files[person_idx])} frame files")
    
    return all_persons_files      

In [6]:
frames_root_path = "/Users/allarviinamae/EduWorkspace/pose-estimation-jupyter/raw-keypoints/backflip-1-allar"
frame_data_path = f"{frames_root_path}/backflip-1-allar.mov-[frame_idx]-[person_idx].csv"

print(f"Frame data path={frame_data_path}")
        
input_files = [Path(f).abspath() for f in glob(frames_root_path + '/*')]

input_files_count = len(input_files)
print(f"Root path includes {input_files_count} files")

all_persons_files = get_person_files(input_files, frame_data_path)

# Collecting frame data for the person with index 0
person_idx_to_collect = 0
person_frame_files = all_persons_files[person_idx_to_collect]

frame_data = [get_frame_data(frame_file) for frame_file in person_frame_files]
 
frame_count = len(frame_data)
print(f"Imported data for {frame_count} frames")

Frame data path=/Users/allarviinamae/EduWorkspace/pose-estimation-jupyter/raw-keypoints/backflip-1-allar/backflip-1-allar.mov-[frame_idx]-[person_idx].csv
Root path includes 87 files
Found 3 persons
Person 0 has 83 frame files
Person 1 has 3 frame files
Person 2 has 1 frame files
Imported data for 83 frames


## Define some functions

In [7]:
def get_min_recognized_coord(df):
    min_recognized = 9999
    
    for coord in df.iloc[:, 0]:
        if coord < min_recognized and coord != 0:
            min_recognized = coord
            
    return min_recognized

def get_xmin_xmax(df):
    x_min = get_min_recognized_coord(df)
    x_max = max(df.iloc[:, 0])
    
    return (x_min, x_max)

def get_minmax_avg(minmax):
    return (minmax[0] + minmax[1]) / 2

## Define min max functions 

In [45]:
def get_frame_wo_x_zeros(frame):
    return frame[frame['x'] != 0]

def get_frame_wo_y_zeros(frame):
    return frame[frame['y'] != 0]

def get_max_x_coord(frame_data):
    return max([max(frame.iloc[:, 0]) for frame in frame_data])

def get_min_x_coord(frame_data):    
    return min([min(get_frame_wo_x_zeros(frame).iloc[:, 0]) for frame in frame_data])

def get_max_y_coord(frame_data):
    return max([max(frame.iloc[:, 1]) for frame in frame_data])

def get_min_y_coord(frame_data):    
    return min([min(get_frame_wo_y_zeros(frame).iloc[:, 1]) for frame in frame_data])

def get_flipped(df_orig):
    df = df_orig.copy()
    
    df['y'] *= -1
    
    max_y = get_max_y_coord(frame_data)
    min_y = get_min_y_coord(frame_data)
    
    df['y'] += max_y + min_y
    
    return df

In [46]:
def get_body_part(body_connection, df_orig):
    frm = body_connection[0]
    to = body_connection[1]
    
    df = get_flipped(df_orig)
    
    if df.iloc[to, 0] == 0 or df.iloc[to, 1] == 0:
        return
    if df.iloc[frm, 0] == 0 or df.iloc[frm, 1] == 0:
        return
    
    x0 = df.iloc[frm, 0]
    y0 = df.iloc[frm, 1]
    
    x1 = df.iloc[to, 0]
    y1 = df.iloc[to, 1]
    
    return go.Scatter(x=[x0, x1], y=[y0, y1], name=body_connection[2])

def get_all_body_parts(df):
    body_connections = [(0, 1, 'Neck'),
                        (15, 0, 'REye'),
                        (16, 0, 'LEye'),
                        (1, 8, 'Spine'),
                        (1, 2, 'RShoulder'),
                        (2, 3, 'RArm'),
                        (3, 4, 'RForearm'),
                        (1, 5, 'LShoulder'),
                        (5, 6, 'LArm'),
                        (6, 7, 'LForearm'),
                        (8, 9, 'RHip'),
                        (9, 10, 'RThigh'),
                        (10, 11, 'RCalf'),
                        (11, 22, 'RFoot'),
                        (8, 12, 'LHip'),
                        (12, 13, 'LThigh'),
                        (13, 14, 'LCalf'),
                        (14, 19, 'LFoot')]
    
    return [get_body_part(body_connection, df) for body_connection in body_connections if get_body_part(body_connection, df) != None]  

def get_frames(df):
    return [go.Frame(data=get_all_body_parts(df[i])) for i in range(1, frame_count)]

def add_body_part(frm, to, df, fig):
    if df.iloc[to, 0] == 0 or df.iloc[to, 1] == 0:
        return
    if df.iloc[frm, 0] == 0 or df.iloc[frm, 1] == 0:
        return
    
    fig.add_shape(
        go.layout.Shape(
            type="line",
            x0=df.iloc[frm, 0],
            y0=df.iloc[frm, 1],
            x1=df.iloc[to, 0],
            y1=df.iloc[to, 1],
            line=dict(
                color="RoyalBlue",
            ),
        ))
    
# Body Parts
def add_body_parts(df_orig, fig):
    df = get_flipped(df_orig)
    
    add_body_part(0, 1, df, fig)
    add_body_part(1, 8, df, fig)

    add_body_part(1, 2, df, fig)
    add_body_part(2, 3, df, fig)
    add_body_part(3, 4, df, fig)

    add_body_part(1, 5, df, fig)
    add_body_part(5, 6, df, fig)
    add_body_part(6, 7, df, fig)

    add_body_part(8, 9, df, fig)
    add_body_part(9, 10, df, fig)
    add_body_part(10, 11, df, fig)

    add_body_part(8, 12, df, fig)
    add_body_part(12, 13, df, fig)
    add_body_part(13, 14, df, fig)
    
def add_all_body_parts():
    skips = 5
    skip = skips
    
    for i, frame in enumerate(frame_data):
        if skip > 0 and i != 1 and i != 60:
            skip = skip - 1
            continue
        else:
            skip = skips
    
        print(f"Adding frame {i}")
        
        add_body_parts(frame, fig)

In [47]:
def render_frame(frame):
    fig = go.Figure()
    fig.update_xaxes(range=[0, 2500])
    fig.update_yaxes(range=[0, 1500])
        
    add_body_parts(frame, fig)

    fig.show()
        
render_frame(frame_data[5])

In [48]:
max_x_coord = get_max_x_coord(frame_data)
min_x_coord = get_min_x_coord(frame_data)

max_y_coord = get_max_y_coord(frame_data)
min_y_coord = get_min_y_coord(frame_data)

print(f"x max={max_x_coord}, x min={min_x_coord}, y max={max_y_coord}, y min={min_y_coord}")

x max=1298.431640625, x min=854.0137329101561, y max=971.708740234375, y min=224.18251037597656


In [50]:
def frame_args(duration):
    return {
            "frame": {"duration": duration},
            "mode": "immediate",
            "fromcurrent": True,
            "transition": {"duration": duration, "easing": "linear"},
        }

def get_animated_figure(frame_data, max_x_coord, max_y_coord):
    fig = go.Figure(
        data = get_all_body_parts(frame_data[0]),
        layout = go.Layout(
            xaxis=dict(range=[0, max_x_coord + 50], autorange=False),
            yaxis=dict(range=[0, max_y_coord + 50], autorange=False),
            updatemenus=[dict(
                type="buttons",
                buttons=[dict(label="Play",
                              method="animate",
                              args=[None, frame_args(50)]
                             ),
                         dict(label="Pause",
                              method="animate",
                              args=[[None], frame_args(0)]
                             )
                        ]
            )]
        ),
        frames=get_frames(frame_data)
    )
    
    return fig

fig = get_animated_figure(frame_data, max_x_coord, max_y_coord)
fig.show()

## Here we start to modify our data

In [101]:
def get_body_part_y_data(frame, body_part_nr=0):
    return float(frame.iloc[body_part_nr, 1:2])

def get_body_part_x_data(frame, body_part_nr=0):
    return float(frame.iloc[body_part_nr, 0:1])

def show_body_part_trajectory(frame_data, body_part_nr=0, coord="Y"):
    if coord == "Y":
        body_part_data = [[idx, get_body_part_y_data(frame, body_part_nr)] for idx, frame in enumerate(frame_data)]
    elif coord == "X":
        body_part_data = [[idx, get_body_part_x_data(frame, body_part_nr)] for idx, frame in enumerate(frame_data)]

    df = pd.DataFrame(body_part_data, columns = ['Frames', 'YValues']) 

    fig = px.line(df, x='Frames', y='YValues')
    fig.show()

# 0 = nose
# 8 = mid hip
show_body_part_trajectory(frame_data, body_part_nr=19, coord="Y")

In [62]:
show_body_part_trajectory(frame_data, body_part_nr=0, coord="X")

In [108]:
fixed_frame_data = [frame.copy() for frame in frame_data]

In [109]:
def get_new_body_part_data(body_part_data):
    new_body_part_data = []
    
    for idx, body_part in enumerate(body_part_data):
        if idx == 0:
            new_body_part_data.append(body_part)
            continue
            
        previous_body_part = new_body_part_data[idx - 1]
        
        print(f"Body part frame={idx} {body_part} - {previous_body_part} = {body_part - previous_body_part} is {body_part == 0}")
        
        if body_part == 0:
            new_body_part_data.append(previous_body_part)
        else:
            new_body_part_data.append(body_part)
            
    return new_body_part_data

def fix_body_part_data(frame_data, body_part_nr=0):    
    body_part_y_data = [get_body_part_y_data(frame, body_part_nr) for idx, frame in enumerate(frame_data)]
    body_part_x_data = [get_body_part_x_data(frame, body_part_nr) for idx, frame in enumerate(frame_data)]
    
    new_body_part_y_data = get_new_body_part_data(body_part_y_data)
    new_body_part_x_data = get_new_body_part_data(body_part_x_data)
      
    for idx, frame in enumerate(frame_data):
        frame.iloc[body_part_nr, 0:1] = new_body_part_x_data[idx]
        frame.iloc[body_part_nr, 1:2] = new_body_part_y_data[idx]
        
        frame_data[idx] = frame
        
    return frame_data
    

for idx in range(0, 26):
    fixed_frame_data = fix_body_part_data(fixed_frame_data, idx)

Body part frame=1 380.16903686523443 - 382.9988403320313 = -2.829803466796875 is False
Body part frame=2 380.1050415039063 - 380.16903686523443 = -0.063995361328125 is False
Body part frame=3 380.13162231445307 - 380.1050415039063 = 0.026580810546761313 is False
Body part frame=4 380.1937255859375 - 380.13162231445307 = 0.06210327148443184 is False
Body part frame=5 383.1523742675781 - 380.1937255859375 = 2.958648681640625 is False
Body part frame=6 388.8705749511719 - 383.1523742675781 = 5.71820068359375 is False
Body part frame=7 392.0622253417969 - 388.8705749511719 = 3.191650390625 is False
Body part frame=8 406.55572509765625 - 392.0622253417969 = 14.493499755859375 is False
Body part frame=9 412.6163330078125 - 406.55572509765625 = 6.06060791015625 is False
Body part frame=10 427.24978637695307 - 412.6163330078125 = 14.633453369140568 is False
Body part frame=11 439.14736938476557 - 427.24978637695307 = 11.8975830078125 is False
Body part frame=12 456.73730468750006 - 439.1473693

Body part frame=60 1160.113037109375 - 1163.1964111328125 = -3.0833740234375 is False
Body part frame=61 1139.619873046875 - 1160.113037109375 = -20.4931640625 is False
Body part frame=62 0.0 - 1139.619873046875 = -1139.619873046875 is True
Body part frame=63 1163.044677734375 - 1139.619873046875 = 23.4248046875 is False
Body part frame=64 0.0 - 1163.044677734375 = -1163.044677734375 is True
Body part frame=65 1183.7275390625002 - 1163.044677734375 = 20.682861328125227 is False
Body part frame=66 0.0 - 1183.7275390625002 = -1183.7275390625002 is True
Body part frame=67 0.0 - 1183.7275390625002 = -1183.7275390625002 is True
Body part frame=68 1171.9013671875 - 1183.7275390625002 = -11.826171875000227 is False
Body part frame=69 0.0 - 1171.9013671875 = -1171.9013671875 is True
Body part frame=70 0.0 - 1171.9013671875 = -1171.9013671875 is True
Body part frame=71 1180.706787109375 - 1171.9013671875 = 8.805419921875 is False
Body part frame=72 0.0 - 1180.706787109375 = -1180.706787109375 i

Body part frame=67 1145.4461669921875 - 1160.0399169921873 = -14.593749999999773 is False
Body part frame=68 0.0 - 1145.4461669921875 = -1145.4461669921875 is True
Body part frame=69 0.0 - 1145.4461669921875 = -1145.4461669921875 is True
Body part frame=70 1189.6412353515625 - 1145.4461669921875 = 44.195068359375 is False
Body part frame=71 1189.5291748046877 - 1189.6412353515625 = -0.11206054687477263 is False
Body part frame=72 1186.736328125 - 1189.5291748046877 = -2.7928466796877274 is False
Body part frame=73 1195.462646484375 - 1186.736328125 = 8.726318359375 is False
Body part frame=74 1201.3812255859375 - 1195.462646484375 = 5.9185791015625 is False
Body part frame=75 1266.1094970703125 - 1201.3812255859375 = 64.728271484375 is False
Body part frame=76 1260.1829833984375 - 1266.1094970703125 = -5.926513671875 is False
Body part frame=77 1195.4481201171875 - 1260.1829833984375 = -64.73486328125 is False
Body part frame=78 1186.63720703125 - 1195.4481201171875 = -8.8109130859375 

Body part frame=75 1186.689453125 - 1186.61865234375 = 0.07080078125 is False
Body part frame=76 1080.6962890625 - 1186.689453125 = -105.9931640625 is False
Body part frame=77 1139.4605712890625 - 1080.6962890625 = 58.7642822265625 is False
Body part frame=78 1142.43603515625 - 1139.4605712890625 = 2.9754638671875 is False
Body part frame=79 1142.6153564453123 - 1142.43603515625 = 0.17932128906227263 is False
Body part frame=80 1142.4814453125 - 1142.6153564453123 = -0.13391113281227263 is False
Body part frame=81 1136.5020751953123 - 1142.4814453125 = -5.979370117187727 is False
Body part frame=82 1119.03125 - 1136.5020751953123 = -17.470825195312273 is False
Body part frame=1 865.8524780273436 - 868.8168334960938 = -2.9643554687501137 is False
Body part frame=2 865.8436889648436 - 865.8524780273436 = -0.0087890625 is False
Body part frame=3 868.7094726562499 - 865.8436889648436 = 2.86578369140625 is False
Body part frame=4 865.8683471679686 - 868.7094726562499 = -2.84112548828125 is 

Body part frame=82 1166.054443359375 - 1171.9183349609375 = -5.8638916015625 is False
Body part frame=1 880.5611572265626 - 880.6378173828125 = -0.07666015624988631 is False
Body part frame=2 880.5003662109375 - 880.5611572265626 = -0.06079101562511369 is False
Body part frame=3 883.3909912109375 - 880.5003662109375 = 2.890625 is False
Body part frame=4 883.5078735351562 - 883.3909912109375 = 0.11688232421875 is False
Body part frame=5 886.43017578125 - 883.5078735351562 = 2.92230224609375 is False
Body part frame=6 889.4316406249999 - 886.43017578125 = 3.0014648437498863 is False
Body part frame=7 901.0586547851561 - 889.4316406249999 = 11.62701416015625 is False
Body part frame=8 904.1295776367188 - 901.0586547851561 = 3.0709228515626137 is False
Body part frame=9 907.0354614257811 - 904.1295776367188 = 2.9058837890623863 is False
Body part frame=10 912.9363403320311 - 907.0354614257811 = 5.90087890625 is False
Body part frame=11 921.6937866210936 - 912.9363403320311 = 8.757446289062

Body part frame=1 0.0 - 0.0 = 0.0 is True
Body part frame=2 0.0 - 0.0 = 0.0 is True
Body part frame=3 0.0 - 0.0 = 0.0 is True
Body part frame=4 0.0 - 0.0 = 0.0 is True
Body part frame=5 0.0 - 0.0 = 0.0 is True
Body part frame=6 0.0 - 0.0 = 0.0 is True
Body part frame=7 0.0 - 0.0 = 0.0 is True
Body part frame=8 0.0 - 0.0 = 0.0 is True
Body part frame=9 0.0 - 0.0 = 0.0 is True
Body part frame=10 0.0 - 0.0 = 0.0 is True
Body part frame=11 0.0 - 0.0 = 0.0 is True
Body part frame=12 0.0 - 0.0 = 0.0 is True
Body part frame=13 0.0 - 0.0 = 0.0 is True
Body part frame=14 0.0 - 0.0 = 0.0 is True
Body part frame=15 0.0 - 0.0 = 0.0 is True
Body part frame=16 0.0 - 0.0 = 0.0 is True
Body part frame=17 0.0 - 0.0 = 0.0 is True
Body part frame=18 0.0 - 0.0 = 0.0 is True
Body part frame=19 0.0 - 0.0 = 0.0 is True
Body part frame=20 0.0 - 0.0 = 0.0 is True
Body part frame=21 0.0 - 0.0 = 0.0 is True
Body part frame=22 0.0 - 0.0 = 0.0 is True
Body part frame=23 0.0 - 0.0 = 0.0 is True
Body part frame=24 0

Body part frame=1 930.6491088867186 - 930.5694580078125 = 0.07965087890613631 is False
Body part frame=2 930.6986083984376 - 930.6491088867186 = 0.049499511718977374 is False
Body part frame=3 936.4089355468751 - 930.6986083984376 = 5.7103271484375 is False
Body part frame=4 936.4660034179689 - 936.4089355468751 = 0.05706787109375 is False
Body part frame=5 939.40087890625 - 936.4660034179689 = 2.9348754882811363 is False
Body part frame=6 945.2562255859376 - 939.40087890625 = 5.855346679687614 is False
Body part frame=7 951.1460571289064 - 945.2562255859376 = 5.88983154296875 is False
Body part frame=8 954.0111694335936 - 951.1460571289064 = 2.8651123046872726 is False
Body part frame=9 954.229248046875 - 954.0111694335936 = 0.2180786132813637 is False
Body part frame=10 960.0745239257811 - 954.229248046875 = 5.845275878906136 is False
Body part frame=11 962.9454956054688 - 960.0745239257811 = 2.8709716796876137 is False
Body part frame=12 962.9832763671876 - 962.9454956054688 = 0.037

Body part frame=1 909.9897460937501 - 912.8760375976561 = -2.8862915039060226 is False
Body part frame=2 909.9223022460936 - 909.9897460937501 = -0.06744384765647737 is False
Body part frame=3 907.0387573242189 - 909.9223022460936 = -2.8835449218747726 is False
Body part frame=4 906.9136352539062 - 907.0387573242189 = -0.1251220703126137 is False
Body part frame=5 906.95703125 - 906.9136352539062 = 0.04339599609375 is False
Body part frame=6 906.9890136718751 - 906.95703125 = 0.03198242187511369 is False
Body part frame=7 907.072021484375 - 906.9890136718751 = 0.08300781249988631 is False
Body part frame=8 910.0775146484376 - 907.072021484375 = 3.0054931640626137 is False
Body part frame=9 921.7189331054689 - 910.0775146484376 = 11.64141845703125 is False
Body part frame=10 927.5827026367188 - 921.7189331054689 = 5.863769531249886 is False
Body part frame=11 927.5996093750001 - 927.5827026367188 = 0.016906738281363687 is False
Body part frame=12 927.6070556640625 - 927.5996093750001 = 

IndexError: single positional indexer is out-of-bounds

In [112]:
show_body_part_trajectory(fixed_frame_data, body_part_nr=3, coord="Y")

In [111]:
fig = get_animated_figure(fixed_frame_data, max_x_coord, max_y_coord)
fig.show()